# Prepare synthetic data (Vitals)
It is going to be much easier to develop if you have realistic synthetic data.
Here we take a sql query that generates a single tabular output.
We run that query against the live identifiable data once.
We then use the [Synthetic Data Vault](https://sdv.dev/SDV/index.html) to prepare a synthetic model of those data.
The code below serves as a vignette for that process but will need adjusting to match the exact contents of the original query.

More complex examples that include multiple tables with joins and dependencies are also possible.

This notebook should be run interactively just once

## Set-up, query and return the data as datafame
The query lives in `./src/api/sitrep/` where `.` represents the project root.
If you run this JupyterNotebook using the local **Makefile** and `make run` then that query will be automatically copied here.

So first steps should be (from _this_ directory)
```sh
make mock1build
make mock2copyin
make mock3run
```
then navigate to http://uclvlddpragae07:8091/lab/tree/steve/work/synth_test_data_vitals.ipynb

In [73]:
import os
import pandas as pd
import numpy as np
import requests

from pathlib import Path
import sqlalchemy as sa

In [74]:
# Construct the PostgreSQL connection
emap_host = os.getenv('EMAP_DB_HOST')
emap_user = os.getenv('EMAP_DB_USER')
emap_passwd = os.getenv('EMAP_DB_PASSWORD')

emapdb_engine = sa.create_engine(f'postgresql://{emap_user}:{emap_passwd}@{emap_host}:5432/uds')

In [75]:
# Read the sql file into a query 'q' and the query into a dataframe
q = Path('vitals.sql').read_text()

In [76]:
df = pd.read_sql_query(sa.text(q), emapdb_engine)

Now rather than using a SQL query, this time we are going to use an existing API to populate a dataframe

In [77]:
df.shape

(4085, 16)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4085 entries, 0 to 4084
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   visit_observation_id  4085 non-null   int64              
 1   ob_tail_i             4085 non-null   int64              
 2   observation_datetime  4085 non-null   datetime64[ns, UTC]
 3   id_in_application     4085 non-null   object             
 4   value_as_real         2137 non-null   float64            
 5   value_as_text         1939 non-null   object             
 6   unit                  536 non-null    object             
 7   mrn                   4085 non-null   object             
 8   lastname              4085 non-null   object             
 9   firstname             4085 non-null   object             
 10  sex                   4085 non-null   object             
 11  date_of_birth         4085 non-null   object             
 12  bed_ad

## Generate a synthetic version of the real data

Use the table above to generate the metadata you need for the synthetic data

https://sdv.dev/SDV/user_guides/relational/relational_metadata.html#relational-metadata

In [79]:
# Minimal imports
from sdv import Metadata, SDV
from sdv.constraints import FixedCombinations

SDV does not handle timezones nicely so remove

In [80]:
# PostgreSQL returns datetimes with tz info which sdv does not seem to be able to handle
def remove_timezone(df, col_name: str) -> pd.DataFrame:
    """sdv does not like timezones"""
    df[col_name] = pd.to_datetime(df[col_name], utc=True).dt.tz_localize(None)    
    return df

In [81]:
tz_cols = ['observation_datetime', 'bed_admit_dt']
for col in tz_cols:
    print(col)
    df = remove_timezone(df, col)

observation_datetime
bed_admit_dt


### Define PII that must be faked and not modelled

Define fields that contain PII and need faking (see the sketchy documentation [here](https://sdv.dev/SDV/developer_guides/sdv/metadata.html?highlight=pii#categorical-fields-data-anonymization) and the [Faker Documentation](https://faker.readthedocs.io/en/master/providers.html) for a full list of providers. Here is a brief example that specifies Fakers for [name](https://faker.readthedocs.io/en/master/providers/faker.providers.person.html#faker.providers.person.Provider.name) and [date of birth](https://faker.readthedocs.io/en/master/providers/faker.providers.date_time.html#faker.providers.date_time.Provider.date_of_birth). Note that you must pass arguments to a faker as a list.

NB: sdv also doesn't always recognise the columns correctly. Here we specify data_of_birth explicitly as a date whilst working on the larger task of defining columns that contain PII. See [field details](https://sdv.dev/SDV/developer_guides/sdv/metadata.html#field-details)

Now a full specification for the Vitals data

In [82]:
fields = {
    'visit_observation_id' : {
        'type': 'id',
        'subtype': 'integer',
    },
    'date_of_birth': {
        'type': 'datetime',
        'format': '%Y-%m-%d',
        'pii': True,
        # the 'pii_category' key defines the Faker function name (method)
        'pii_category': "date_of_birth", 
    },
    'lastname': {
        'type': 'categorical',
        'pii': True,
        'pii_category': 'last_name'
    },
    'firstname': {
        'type': 'categorical',
        'pii': True,
        'pii_category': 'first_name'
    },
    'mrn': {
        'type': 'categorical',
        'pii': True,
        'pii_category': ['random_number', 8 ]
    },
}

Prepare the constraints

In [108]:
fixed_obs_constraint = FixedCombinations(
    column_names=['id_in_application', 'value_as_text', 'value_as_real', 'unit'],
    handling_strategy='reject_sampling',  # transform strategy fails ?b/c data types vary
)

Prepare the metadata

In [109]:
metadata = Metadata()
metadata.add_table(
    name='tabpid',
    data=df,
    fields_metadata=fields,
    primary_key = 'visit_observation_id',
    constraints=[fixed_obs_constraint], # be patient; reject_sampling is slow
)

In [110]:
# Inspect the conversion that metadata.add_table did to the dataframe that you loaded
metadata.get_table_meta('tabpid')

{'fields': {'visit_observation_id': {'type': 'id', 'subtype': 'integer'},
  'date_of_birth': {'type': 'datetime',
   'format': '%Y-%m-%d',
   'pii': True,
   'pii_category': 'date_of_birth'},
  'lastname': {'type': 'categorical',
   'pii': True,
   'pii_category': 'last_name'},
  'firstname': {'type': 'categorical',
   'pii': True,
   'pii_category': 'first_name'},
  'mrn': {'type': 'categorical',
   'pii': True,
   'pii_category': ['random_number', 8]},
  'ob_tail_i': {'type': 'numerical', 'subtype': 'integer'},
  'observation_datetime': {'type': 'datetime'},
  'id_in_application': {'type': 'categorical'},
  'value_as_real': {'type': 'numerical', 'subtype': 'float'},
  'value_as_text': {'type': 'categorical'},
  'unit': {'type': 'categorical'},
  'sex': {'type': 'categorical'},
  'bed_admit_dt': {'type': 'datetime'},
  'dept_name': {'type': 'categorical'},
  'room_name': {'type': 'categorical'},
  'bed_hl7': {'type': 'categorical'}},
 'constraints': [{'constraint': 'sdv.constraints.ta

Prepare the table(s)

In [111]:
tables = dict(tabpid=df)

Fit the model

In [112]:
sdv = SDV()
sdv.fit(metadata, tables)

In [113]:
dfs = sdv.sample_all()['tabpid']

In [114]:
dfs.loc[:5]

,visit_observation_id,date_of_birth,lastname,firstname,mrn,ob_tail_i,observation_datetime,id_in_application,value_as_real,value_as_text,unit,sex,bed_admit_dt,dept_name,room_name,bed_hl7
0,0,1965-03-08,Carroll,Kelsey,26856213,1,2022-06-26 09:30:13,6466,NaN,A,NaN,M,2022-07-07 07:51:59,UCH T03 INTENSIVE CARE,BY05,BY01-05
1,1,1984-01-27,Jacobson,Jonathan,38519026,1,2022-06-26 09:06:04,6466,NaN,A,NaN,F,2022-07-04 15:46:07,UCH T12 SOUTH (T12S),BY13,BY10-47
2,2,1915-09-07,Jordan,Rebecca,12042535,1,2022-06-26 08:20:30,8,67.0,NaN,NaN,M,2022-07-08 18:38:10,UCH T16 NORTH (T16N),BY01,BY09-45
3,3,1915-08-06,Taylor,Jonathan,5592674,1,2022-06-26 09:23:29,28315,NaN,NaN,NaN,M,2022-06-28 15:04:20,UCH P02 ENDOSCOPY,POOL ROOM,BY11-55
4,4,1942-09-02,Duncan,Matthew,7517646,1,2022-06-26 08:19:44,6466,NaN,A,NaN,F,2022-06-10 09:29:38,UCH T10 NORTH (T10N),MAJ-CHAIR,SR32-32
5,5,1989-02-21,Lyons,Michael,10532857,1,2022-06-26 09:53:25,6466,NaN,A,NaN,M,2022-06-24 22:04:02,UCH T03 INTENSIVE CARE,SR03,BY02-05


In [115]:
dfs['id_in_application'].value_counts().sort_index()

10              57
28315          396
28316           66
3040109304     806
5              406
6              319
6466          1480
8              333
9              222
Name: id_in_application, dtype: int64

Inspect the original data

In [116]:
df['id_in_application'].value_counts().sort_index()

10            536
28315         424
28316          28
3040109304    498
5             481
6             541
6466          513
8             534
9             530
Name: id_in_application, dtype: int64

In [117]:
# df.head()

Inspect the synthetic data

Finally transform admission_age_years to match the fake DoB

### Save the synthetic data

Options
- save the model and not the synthetic data (but then you need *sdv* to run the model)
- save the data (need some care with type conversions if you use csv etc.)

In [118]:
model.save('mock_vitals.pkl')

In [119]:
dfs.to_hdf('mock_vitals.h5', 'data')

/tmp/ipykernel_291/1530517875.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['lastname', 'firstname', 'mrn', 'id_in_application', 'value_as_text',
       'unit', 'sex', 'dept_name', 'room_name', 'bed_hl7'],
      dtype='object')]

  dfs.to_hdf('mock_vitals.h5', 'data')


In [120]:
pd.read_hdf('mock_vitals.h5', 'data')

,visit_observation_id,date_of_birth,lastname,firstname,mrn,ob_tail_i,observation_datetime,id_in_application,value_as_real,value_as_text,unit,sex,bed_admit_dt,dept_name,room_name,bed_hl7
0,0,1965-03-08,Carroll,Kelsey,26856213,1,2022-06-26 09:30:13,6466,NaN,A,NaN,M,2022-07-07 07:51:59,UCH T03 INTENSIVE CARE,BY05,BY01-05
1,1,1984-01-27,Jacobson,Jonathan,38519026,1,2022-06-26 09:06:04,6466,NaN,A,NaN,F,2022-07-04 15:46:07,UCH T12 SOUTH (T12S),BY13,BY10-47
2,2,1915-09-07,Jordan,Rebecca,12042535,1,2022-06-26 08:20:30,8,67.0,NaN,NaN,M,2022-07-08 18:38:10,UCH T16 NORTH (T16N),BY01,BY09-45
3,3,1915-08-06,Taylor,Jonathan,5592674,1,2022-06-26 09:23:29,28315,NaN,NaN,NaN,M,2022-06-28 15:04:20,UCH P02 ENDOSCOPY,POOL ROOM,BY11-55
4,4,1942-09-02,Duncan,Matthew,7517646,1,2022-06-26 08:19:44,6466,NaN,A,NaN,F,2022-06-10 09:29:38,UCH T10 NORTH (T10N),MAJ-CHAIR,SR32-32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4080,4080,2018-06-05,Drake,Kathleen,70624460,1,2022-06-26 07:49:30,3040109304,NaN,Room air,NaN,M,2022-06-07 07:11:49,UCH T01 ACUTE MEDICAL,BY11,BY04-24G
4081,4081,1923-10-25,Hoffman,Jacqueline,72676299,1,2022-06-26 08:38:40,8,NaN,NaN,NaN,M,2022-07-14 01:09:04,UCH EMERGENCY DEPT,SR08,BY02-05
4082,4082,1939-11-04,Watts,Nathan,42941843,1,2022-06-26 08:53:26,28315,NaN,1,NaN,M,2022-05-23 06:48:29,UCH T16 SOUTH WINTER,BY08,BY12-59
4083,4083,1957-05-28,Luna,Katrina,98808371,1,2022-06-26 08:12:53,9,27.0,NaN,NaN,M,2022-06-06 19:12:13,UCH T03 INTENSIVE CARE,POOL ROOM,WAIT
